In [4]:
!pip install selenium
!pip install bs4
!pip install webdriver-manager
!pip install pandas
!pip install lxml


     ---------------------------------------- 11.0/11.0 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 14.7/14.7 MB 1.6 MB/s eta 0:00:00

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pysd 3.0.1 requires chardet, which is not installed.
pysd 3.0.1 requires openpyxl, which is not installed.
pysd 3.0.1 requires parsimonious, which is not installed.
pysd 3.0.1 requires progressbar2, which is not installed.
pysd 3.0.1 requires regex, which is not installed.
pysd 3.0.1 requires scipy, which is not installed.
pysd 3.0.1 requires xarray, which is not installed.
pysd 3.0.1 requires xlrd, which is not installed.


In [84]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

def modifyPageWithDriver(page):
    try:
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(page)

        # to click on input field
        driver.find_element(By.XPATH,"//*[@id='ctl00_CPH1_BtnTipoGobierno']").click()

        time.sleep(2)

        # to click on the element(E: GOBIERNO NACIONAL) found
        driver.find_element(By.XPATH,"//*[@id='ctl00_CPH1_RptData_ctl01_TD0']").click()

        time.sleep(2)

        # to click on input field
        driver.find_element(By.XPATH,"//*[@id='ctl00_CPH1_BtnSector']").click()

        time.sleep(2)

        page_source = driver.page_source

        driver.close()
        return page_source
    except:
        raise Exception("Error with selenium")


In [85]:
def findUsefulDataSoup(page_source):
    try:
        soup = BeautifulSoup(page_source, 'lxml')
        sectors_useful_data = []
        table_data = soup.select(".Data")
        sectors_selector = table_data[0].find_all('tr')
        for sector in sectors_selector:
            sector_columns = sector.find_all("td")
            sector_info = []
            sector_info.append(sector_columns[1].string.strip())
            sector_info.append(sector_columns[3].string.strip())
            sector_info.append(sector_columns[7].string.strip())
            sector_info.append(sector_columns[9].string.strip())
            sectors_useful_data.append(sector_info)
        return sectors_useful_data
    except:
        raise Exception("Error parsing html data")

In [86]:
def creatingCSV(data):
    try:
      ## LEYENDO CSV EXISTENTE  
      df = pd.read_csv('./consulta-amigable.csv', index_col=False, header=0, ) 
      print("Leyendo CSV EXISTENTE")
    except:
      ## CREAR MEDIANTE DATAFRAME
      df = pd.DataFrame({'Sector':[],
                       'PIM':[],
                       'Devengado':[],
                        '% Avance':[]})
      print("NO SE ENCONTRO CSV EXISTENTE")
    for index,sector in enumerate(sectors_useful_data):
        df.loc[index,['Sector', 'PIM', 'Devengado', '% Avance']] = sector

    df.to_csv('./consulta-amigable.csv',index=False)

In [87]:
def scrapCA():
    try:
        url = "https://apps5.mineco.gob.pe/transparencia/Navegador/Navegar_7.aspx"
        page_source =modifyPageWithDriver(url)
        data = findUsefulDataSoup(page_source)
        creatingCSV(data)
        return 1
    except:
        return 0
        

In [88]:
print(scrapCA())

Leyendo CSV EXISTENTE
1
